In [7]:
# Doppio Basics
#Import basic packages:
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import pandas as pd

#import stuff for plotting
import matplotlib.ticker as ticker

import matplotlib.gridspec as gridspec
import cmocean #install new package from conda-forge
import cmocean.cm as cmo #these are nice colormaps for matplotlib
import matplotlib.dates as mdates
from matplotlib.pyplot import figure
from matplotlib.pyplot import cm
#these are to make maps:
import cartopy.crs as ccrs
from cartopy.feature import ShapelyFeature
import cartopy.feature as cfeature
from cartopy import config

%matplotlib inline
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER


In [8]:

## Load the data (the actual URL to look at all the data is here: https://tds.marine.rutgers.edu/thredds/roms/doppio/catalog.html)
num_time_chunks = 1  # Number of chunks along the 'time' dimension
num_lat_chunks = 10  # Number of chunks along the 'lat' dimension
num_lon_chunks = 10  # Number of chunks along the 'lon' dimension

# Calculate the size of each chunk along each dimension
# Assuming the dataset has dimensions in bytes (you may need to adjust)
chunk_size_time = '100MB / {}'.format(num_time_chunks)
chunk_size_lat = '100MB / {}'.format(num_lat_chunks)
chunk_size_lon = '100MB / {}'.format(num_lon_chunks)

# Create a dictionary with chunk sizes for each dimension
chunk_sizes = {'time': chunk_size_time, 'lat': chunk_size_lat, 'lon': chunk_size_lon}

url = "https://tds.marine.rutgers.edu/thredds/dodsC/roms/doppio/DopAnV3R3-ini2007_da/avg"
ds = xr.open_dataset(url, chunks=chunk_sizes)
ds


syntax error, unexpected WORD_WORD, expecting SCAN_ATTR or SCAN_DATASET or SCAN_ERROR
context: Request^ Too Large: 50951.02848 Mbytes, max=500.0


<xarray.Dataset>
Dimensions:         (tracer: 2, boundary: 4, s_rho: 40, s_w: 41, eta_rho: 106, xi_rho: 242, eta_u: 106, xi_u: 241, eta_v: 105, xi_v: 242, eta_psi: 105, xi_psi: 241, ocean_time: 6207)
Coordinates:
  * s_rho           (s_rho) float64 -0.9875 -0.9625 -0.9375 ... -0.0375 -0.0125
  * s_w             (s_w) float64 -1.0 -0.975 -0.95 -0.925 ... -0.05 -0.025 0.0
    lon_rho         (eta_rho, xi_rho) float64 dask.array<chunksize=(106, 242), meta=np.ndarray>
    lat_rho         (eta_rho, xi_rho) float64 dask.array<chunksize=(106, 242), meta=np.ndarray>
    lon_u           (eta_u, xi_u) float64 dask.array<chunksize=(106, 241), meta=np.ndarray>
    lat_u           (eta_u, xi_u) float64 dask.array<chunksize=(106, 241), meta=np.ndarray>
    lon_v           (eta_v, xi_v) float64 dask.array<chunksize=(105, 242), meta=np.ndarray>
    lat_v           (eta_v, xi_v) float64 dask.array<chunksize=(105, 242), meta=np.ndarray>
    lon_psi         (eta_psi, xi_psi) float64 dask.array<chunksize=(105, 241), meta=np.ndarray>
    lat_psi         (eta_psi, xi_psi) float64 dask.array<chunksize=(105, 241), meta=np.ndarray>
  * ocean_time      (ocean_time) datetime64[ns] 2007-01-02T12:00:00 ... 2023-...
Dimensions without coordinates: tracer, boundary, eta_rho, xi_rho, eta_u, xi_u, eta_v, xi_v, eta_psi, xi_psi
Data variables: (12/103)
    ntimes          int32 ...
    ndtfast         int32 ...
    dt              float64 ...
    dtfast          float64 ...
    dstart          datetime64[ns] ...
    shuffle         int32 ...
    ...              ...
    salt            (ocean_time, s_rho, eta_rho, xi_rho) float64 dask.array<chunksize=(6207, 40, 106, 242), meta=np.ndarray>
    shflux          (ocean_time, eta_rho, xi_rho) float64 dask.array<chunksize=(6207, 106, 242), meta=np.ndarray>
    ssflux          (ocean_time, eta_rho, xi_rho) float64 dask.array<chunksize=(6207, 106, 242), meta=np.ndarray>
    swrad_daily     (ocean_time, eta_rho, xi_rho) float64 dask.array<chunksize=(6207, 106, 242), meta=np.ndarray>
    sustr           (ocean_time, eta_u, xi_u) float64 dask.array<chunksize=(6207, 106, 241), meta=np.ndarray>
    svstr           (ocean_time, eta_v, xi_v) float64 dask.array<chunksize=(6207, 105, 242), meta=np.ndarray>
Attributes: (12/32)
    file:              doppio_avg_6567.nc
    format:            netCDF-4/HDF5 file
    Conventions:       CF-1.4, SGRID-0.3
    type:              ROMS/TOMS nonlinear model averages file
    title:             DopAnV3R3-ini2007 - ROMS DOPPIO 7km reanalysis (assimi...
    var_info:          ../../Data/varinfo1040t_daily.dat
    ...                ...
    compiler_flags:    -fp-model precise -heap-arrays -ip -O3 -traceback -che...
    tiling:            010x004
    history:           ROMS/TOMS, Version 3.9, Monday - February 19, 2024 - 1...
    ana_file:          ROMS/Functionals/ana_btflux.h
    CPP_options:       DOPPIO, ADD_FSOBC, ADD_M2OBC, ANA_BSFLUX, ANA_BTFLUX, ...
    summary:           doppio

In [9]:

## FOR JUST ONE SINGULAR POINT: 
## Pull out location
my_lon1=-74.1445
my_lat1=39.2714
ds1 = abs(ds.lat_rho-my_lat1)+abs(ds.lon_rho-my_lon1)
a,b = np.unravel_index(ds1.argmin(),ds1.shape)
ds2=ds.sel(eta_rho=a, xi_rho=b)

## Calculate depth (Because ROMS is gridded it does not have real life lat, lon and depth. Here we have to transform the "depth" coordinate from doppio to real world depth)

if ds2.Vtransform == 1:
        Zo_rho = ds2.hc * (ds2.s_rho - ds2.Cs_r) + ds2.Cs_r * ds2.h
        z_rho = Zo_rho + ds2.zeta * (1 + Zo_rho/ds2.h)
elif ds2.Vtransform == 2:
        Zo_rho = (ds2.hc * ds2.s_rho + ds2.Cs_r * ds2.h) / (ds2.hc + ds2.h)
        z_rho = ds2.zeta + (ds2.zeta + ds2.h) * Zo_rho

ds2.coords['z_rho'] = z_rho.transpose()

## now we can call z_rho as our depth coordinate in meters
ds2.coords['z_rho']

<xarray.DataArray 'z_rho' (s_rho: 40, ocean_time: 6207)>
dask.array<transpose, shape=(40, 6207), dtype=float64, chunksize=(40, 6207), chunktype=numpy.ndarray>
Coordinates:
  * s_rho       (s_rho) float64 -0.9875 -0.9625 -0.9375 ... -0.0375 -0.0125
    lon_rho     float64 dask.array<chunksize=(), meta=np.ndarray>
    lat_rho     float64 dask.array<chunksize=(), meta=np.ndarray>
  * ocean_time  (ocean_time) datetime64[ns] 2007-01-02T12:00:00 ... 2023-12-3...
    z_rho       (s_rho, ocean_time) float64 dask.array<chunksize=(40, 6207), meta=np.ndarray>

In [16]:

# MULTIPLE POINTS: 
df = pd.read_excel('/Users/henrysun_1/Desktop/Duke/2024-2025/Summer 2024/fishics/demo/njsites.xls') 
# njsites.xls found in demo folder - modify to my own data (select points)

df.rename(columns={"Unnamed: 0":"objectID","Unnamed: 1":"Lon","Unnamed: 2":"Lat", "Unnamed: 3":"Site Name", "Unnamed: 6":"location"},inplace=True) # Change the column names to Lon/Lat
df.round(4)
df.round({"Lon":4, "Lat":4})
Lon = df.Lon
Lat = df.Lat


## This is where I would use xarray to select the dates you would like so you can subset the data. 
## the variable name is ocean_time and you can even slice the time using something like this: https://docs.xarray.dev/en/v2023.09.0/user-guide/indexing.html (xr.sel)
## Then you can loop through the points to calculate depth in meters, pull out stratification (stratification is the gradient in temperature in a water column profile divided by the gradient in depth (or dT/dz)
## basically just change in temp/change in depth) and temperature for each location:
## REMEMBER TO CHANGE DS IN ALLPNTS FROM DS TO WHATEVER YOU RENAME YOUR TIME SLICED DATASET
ds = ds.sel(ocean_time = '2023-09-12', method = 'nearest')
temp_final=[]
dtdz_final = []
salt_final=[]
ds

<xarray.Dataset>
Dimensions:         (tracer: 2, boundary: 4, s_rho: 40, s_w: 41, eta_rho: 106, xi_rho: 242, eta_u: 106, xi_u: 241, eta_v: 105, xi_v: 242, eta_psi: 105, xi_psi: 241, ocean_time: 1)
Coordinates:
  * s_rho           (s_rho) float64 -0.9875 -0.9625 -0.9375 ... -0.0375 -0.0125
  * s_w             (s_w) float64 -1.0 -0.975 -0.95 -0.925 ... -0.05 -0.025 0.0
    lon_rho         (eta_rho, xi_rho) float64 dask.array<chunksize=(106, 242), meta=np.ndarray>
    lat_rho         (eta_rho, xi_rho) float64 dask.array<chunksize=(106, 242), meta=np.ndarray>
    lon_u           (eta_u, xi_u) float64 dask.array<chunksize=(106, 241), meta=np.ndarray>
    lat_u           (eta_u, xi_u) float64 dask.array<chunksize=(106, 241), meta=np.ndarray>
    lon_v           (eta_v, xi_v) float64 dask.array<chunksize=(105, 242), meta=np.ndarray>
    lat_v           (eta_v, xi_v) float64 dask.array<chunksize=(105, 242), meta=np.ndarray>
    lon_psi         (eta_psi, xi_psi) float64 dask.array<chunksize=(105, 241), meta=np.ndarray>
    lat_psi         (eta_psi, xi_psi) float64 dask.array<chunksize=(105, 241), meta=np.ndarray>
  * ocean_time      (ocean_time) datetime64[ns] 2023-09-12T12:00:00
Dimensions without coordinates: tracer, boundary, eta_rho, xi_rho, eta_u, xi_u, eta_v, xi_v, eta_psi, xi_psi
Data variables: (12/103)
    ntimes          int32 ...
    ndtfast         int32 ...
    dt              float64 ...
    dtfast          float64 ...
    dstart          datetime64[ns] ...
    shuffle         int32 ...
    ...              ...
    salt            (ocean_time, s_rho, eta_rho, xi_rho) float64 dask.array<chunksize=(1, 40, 106, 242), meta=np.ndarray>
    shflux          (ocean_time, eta_rho, xi_rho) float64 dask.array<chunksize=(1, 106, 242), meta=np.ndarray>
    ssflux          (ocean_time, eta_rho, xi_rho) float64 dask.array<chunksize=(1, 106, 242), meta=np.ndarray>
    swrad_daily     (ocean_time, eta_rho, xi_rho) float64 dask.array<chunksize=(1, 106, 242), meta=np.ndarray>
    sustr           (ocean_time, eta_u, xi_u) float64 dask.array<chunksize=(1, 106, 241), meta=np.ndarray>
    svstr           (ocean_time, eta_v, xi_v) float64 dask.array<chunksize=(1, 105, 242), meta=np.ndarray>
Attributes: (12/32)
    file:              doppio_avg_6567.nc
    format:            netCDF-4/HDF5 file
    Conventions:       CF-1.4, SGRID-0.3
    type:              ROMS/TOMS nonlinear model averages file
    title:             DopAnV3R3-ini2007 - ROMS DOPPIO 7km reanalysis (assimi...
    var_info:          ../../Data/varinfo1040t_daily.dat
    ...                ...
    compiler_flags:    -fp-model precise -heap-arrays -ip -O3 -traceback -che...
    tiling:            010x004
    history:           ROMS/TOMS, Version 3.9, Monday - February 19, 2024 - 1...
    ana_file:          ROMS/Functionals/ana_btflux.h
    CPP_options:       DOPPIO, ADD_FSOBC, ADD_M2OBC, ANA_BSFLUX, ANA_BTFLUX, ...
    summary:           doppio

In [21]:
for k in range(6): # we setup our loop here and use k as a placeholder for iterating through our however many points

    Allpnts = abs(ds.lat_rho-Lat[k])+abs(ds.lon_rho-Lon[k]) # I've taken Lat/Lon from the dataframe and put them in your equations, then use k to tell the equation which version of k to use
    a,b = np.unravel_index(Allpnts.argmin(),Allpnts.shape)
    Allpnts=ds.sel(eta_rho=a, xi_rho=b)
    
    if Allpnts.Vtransform == 1:
        Zo_rho = Allpnts.hc * (Allpnts.s_rho - Allpnts.Cs_r) + Allpnts.Cs_r * Allpnts.h
        z_rho = Zo_rho + Allpnts.zeta * (1 + Zo_rho/Allpnts.h)
    elif Allpnts.Vtransform == 2:
        Zo_rho = (Allpnts.hc * Allpnts.s_rho + Allpnts.Cs_r * Allpnts.h) / (Allpnts.hc + Allpnts.h)
        z_rho = Allpnts.zeta + (Allpnts.zeta + Allpnts.h) * Zo_rho
   
    
    Allpnts.coords['z_rho'] = z_rho.transpose()
   
    temp1 = Allpnts.temp
    salt = Allpnts.salt

    l=np.diff(temp1).values
    
    
#     ## ERRORS HERE! 
#     temp_diff = np.diff(temp)/np.diff(np.abs(z_rho)) # Need to take the maximum of this to get max(dT/dz)
#     # salt_diff=np.diff(salt)/np.diff(z_rho)
    
#     tempdtdz1=np.max(-1*temp_diff, axis=1)
#     # saltdtdz1=np.max(-1.*salt_diff, axis=1)

#     #bottom_temp = temp[:,0]
    
#     # plt.plot(tm,temp[:,39])
#     # plt.plot(tm,bottom_temp)

# # I did it!
#     data_xr = xr.DataArray(tempdtdz1, 
#     coords={'ocean_time': ds.ocean_time}, 
#     dims=['ocean_time'])
    
    
#     temp_final.append(temp) #this is an example of how you create a new xarray object, note that above I initialized temp_final as an empty array, and below I use the xarray concat function OUTSIDE of the loop... I don't know why this is how it works :) 
#     # bottom_temp_final.append(bottom_temp)
#     salt_final.append(salt)
#     dtdz_final.append(data_xr)
    

# ## Now you can call this xarray object for temperature to plot temperature at a specific point. You may not want this one, but it should work the same with the other variables
# combined_tempdtdz = xr.concat(dtdz_final,dim='k')
# combined_temp=xr.concat(temp_final,dim='k')
# combined_salt=xr.concat(salt_final,dim='k')
# ## now you can call combined_tempdtdz etc to make plots and look at data! or take averages over a time period. 
# ## These should have z_rho in their coordinates for the depth but let me know if that isn't true. 

syntax error, unexpected WORD_WORD, expecting SCAN_ATTR or SCAN_DATASET or SCAN_ERROR
context: Request^ Too Large: 50951.02848 Mbytes, max=500.0
syntax error, unexpected WORD_WORD, expecting SCAN_ATTR or SCAN_DATASET or SCAN_ERROR
context: Request^ Too Large: 50951.02848 Mbytes, max=500.0
syntax error, unexpected WORD_WORD, expecting SCAN_ATTR or SCAN_DATASET or SCAN_ERROR
context: Request^ Too Large: 50951.02848 Mbytes, max=500.0
syntax error, unexpected WORD_WORD, expecting SCAN_ATTR or SCAN_DATASET or SCAN_ERROR
context: Request^ Too Large: 50951.02848 Mbytes, max=500.0
syntax error, unexpected WORD_WORD, expecting SCAN_ATTR or SCAN_DATASET or SCAN_ERROR
context: Request^ Too Large: 50951.02848 Mbytes, max=500.0
syntax error, unexpected WORD_WORD, expecting SCAN_ATTR or SCAN_DATASET or SCAN_ERROR
context: Request^ Too Large: 50951.02848 Mbytes, max=500.0
syntax error, unexpected WORD_WORD, expecting SCAN_ATTR or SCAN_DATASET or SCAN_ERROR
context: Request^ Too Large: 50951.02848 Mby

RuntimeError: NetCDF: Authorization failure

In [15]:
pip install xlrd



[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
